# Caracterização Climática de São Paulo

Por que interessante: São Paulo é a maior metrópole do Brasil e apresenta um clima subtropical com estações relativamente bem definidas. A cidade é excelente para estudar como a urbanização impacta o microclima, gerando efeitos como as ilhas de calor urbanas. Além disso, a proximidade da Serra do Mar traz eventos de precipitação intensa, influenciando a análise das chuvas e enchentes.
Aspectos climáticos: Verões quentes e chuvosos, invernos mais frios e secos, com maior amplitude térmica anual em comparação a cidades tropicais.

# Definir as estações meteorológicas que possam representar melhor a cidade.

o objetivo desse projeto é caracterizar os microclimas da cidade de são paulo e compará-los afim de verificar se existem diferenças significativas provocadas pelas características dos territórios. A cidade é extensa dividida em cinco regioes norte sul leste oeste cada uma com ocupações distintas.

A primeira etapa é definir quais estacoes meteorológicas de cada regiao serao utilizadas. O INMET (Instituto Nacional de Meteorologia) é a fonte dos dados desse projeto e fornece uma base de dados com as coordenadas de cada estacao no Brasil, assim como os dados coletados de cada estacao.

Para cada regiao serão selecionadas três estacoes, ou seja, sera feitqa uma triangulacao de forma que possa ser representativa.

## Escolha das estacoes

O INMET fornece dois catálogos com suas estacoes - estações automáticas e estações convencionais. Cada catalogo possui cinco dimensões: 

In [3]:
"""
import pandas as pd
import folium

df = pd.read_csv('CatalogoEstaçõesAutomáticas.csv', sep=';')
df['VL_LATITUDE'] = df['VL_LATITUDE'].str.replace(',', '.').astype(float)
df['VL_LONGITUDE'] = df['VL_LONGITUDE'].str.replace(',', '.').astype(float)
mapa = folium.Map(location=[df['VL_LATITUDE'].mean(), df['VL_LONGITUDE'].mean()], zoom_start=6)

for idx, row in df.iterrows():
    folium.Marker(
        location=[row['VL_LATITUDE'], row['VL_LONGITUDE']],
        popup=row.get('DC_NOME', 'CD_ESTACAO'),
        icon=folium.Icon(color='blue')
    ).add_to(mapa)

folium.TileLayer(
    tiles='http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', 
    attr='Google',
    name='Google Satellite',
    overlay=False,
    control=True
).add_to(mapa)
    
"""

"\nimport pandas as pd\nimport folium\n\ndf = pd.read_csv('CatalogoEstaçõesAutomáticas.csv', sep=';')\ndf['VL_LATITUDE'] = df['VL_LATITUDE'].str.replace(',', '.').astype(float)\ndf['VL_LONGITUDE'] = df['VL_LONGITUDE'].str.replace(',', '.').astype(float)\nmapa = folium.Map(location=[df['VL_LATITUDE'].mean(), df['VL_LONGITUDE'].mean()], zoom_start=6)\n\nfor idx, row in df.iterrows():\n    folium.Marker(\n        location=[row['VL_LATITUDE'], row['VL_LONGITUDE']],\n        popup=row.get('DC_NOME', 'CD_ESTACAO'),\n        icon=folium.Icon(color='blue')\n    ).add_to(mapa)\n\nfolium.TileLayer(\n    tiles='http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', \n    attr='Google',\n    name='Google Satellite',\n    overlay=False,\n    control=True\n).add_to(mapa)\n    \n"

In [17]:
import os
import logging
import csv

sourcesPath = os.path.join(os.getcwd(), "sources")

fileCount = 0
representativeDir = None

for root, dirs, files in os.walk(sourcesPath):
    for file in files:
        if not file.startswith("INMET_SE_SP_"):
            filePath = os.path.join(root, file)
            try:
                os.remove(filePath)
            except Exception as e:
                logging.error(f"Erro ao excluir {filePath}: {e}")
    
    if not os.listdir(root):
        try:
            os.rmdir(root)
        except Exception as e:
            logging.error(f"Erro ao excluir o diretório {root}: {e}")
    
    dirFileCount = len(files)
    if dirFileCount > fileCount:
        fileCount = dirFileCount
        representativeDir = root
"""
with open(f"weather_stations_{os.path.basename(most_representative_year)}", mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)"""

'\nwith open(f"weather_stations_{os.path.basename(most_representative_year)}", mode="w", newline="", encoding="utf-8") as csv_file:\n    writer = csv.writer(csv_file)'